# Curve Fit With or Without Constant

When fitting an exponential to the early data for COVID-19 hospitalizations in San Diego, what is the effect of including or not including a constant in the fit function?


In [1]:
import sys
# Install required packages
!{sys.executable} -mpip -q install matplotlib seaborn statsmodels pandas  metapack uncertainties sklearn scipy

%matplotlib inline

import pandas as pd
import numpy as np
import metapack as mp
import rowgenerators as rg
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

from sklearn import  linear_model
#from scipy.stats import weibull_min, lognorm, logistic, norm
from scipy.optimize import curve_fit


import statsmodels.api as sm
import uncertainties.unumpy as unp
import uncertainties as unc


In [2]:
pkg = mp.open_package('http://library.metatab.org/sandiegodata.org-covid19.csv') 
pkg

# COVID-19 Cases For San Diego County
`sandiegodata.org-covid19-1.2.43` Last Update: 2020-04-27T05:49:38

_Cases, deaths, hospitalizations and other data scraped from the San Diego COunty COVID-19 website._


This dataset is manually extracted from the daily updates to the [San Diego
County Coronavirus Disease 2019
(COVID-19)](https://www.sandiegocounty.gov/coronavirus/) web page. The table on
the web page is pulled into a [Google
Spreadsheet](https://docs.google.com/spreadsheets/d/1RkgOAgmZIcq7WdNvxn4dih_-aX6
im3fNd9yrfM1-0Ys/edit?usp=sharing), and this dataset extracts the Summary tab
of the Google spreadsheet.

## Caveats

* The County changes the table format ocassionally, often adding columns, to not all variables are available for all dates. 
* This dataset does not include the information for age groups that is available in the source tables. 
* The `cases` value for March 14 is created by interpolating ( exponential ) from the values before and after it.


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [sd_covid_cases](http://library.metatab.org/sandiegodata.org-covid19-1.2.43/data/sd_covid_cases.csv)**. Notebook for creating dataset.

## References
<ul><li> <strong><a href="https://docs.google.com/spreadsheets/d/1RkgOAgmZIcq7WdNvxn4dih_-aX6im3fNd9yrfM1-0Ys/edit#gid=1588490640">spreadsheet</a></strong>. Google spreadsheet, for humans</li><li> <strong>source</string>, <em>gs:1RkgOAgmZIcq7WdNvxn4dih_-aX6im3fNd9yrfM1-0Ys#1588490640</em>. Machine readable source file</li><ul>

In [3]:
df = pkg.resource('sd_covid_cases').read_csv().fillna(0)
df.drop(columns=['notes'], inplace=True)

df['date'] = pd.to_datetime(df.date)
start_date = df.iloc[0].date
start_cases = df.iloc[0].cases
df['day'] = (df.date - start_date).dt.days

df.rename(columns={'hospitalized': 'hosp'}, inplace=True)




# Setup

First we will create a "true" curve for hospitlaizations from an initial fit to the data. We'll add error to this curve, then fit to it again, to assess the accuracy of the curve fit under different conditions. 



In [7]:
## Fit to hospitalizations to get parameters and relative error

# Here are our fit functions 

def func_exp_c(x, a, b, c):
    """Version of func_exp with a constant"""
    return a * np.exp(b * x) +c 
func_exp_c.popt = (6.20811331, 0.17217422, 0)

def func_exp(x, a, b):
    '''Exponential with no constant'''
    return a * np.exp(b * x) 
func_exp.popt = (6.20811331, 0.17217422)

# Analysis date range
start_date = pd.Timestamp('2020-03-16')
end_date = pd.Timestamp('2020-03-26')
t = df.set_index('date').loc[start_date:end_date]

# Create parameters for an initial curve fit. This will 
# produce a new clean curve that we will use to compare other
# fits to
fit_func = func_exp_c
popt, pcov = curve_fit(fit_func, t.day, t.hosp, p0=fit_func.popt)

y_p = fit_func(t.day,*popt).values # Supposing that the curve fit is the true curve
err_std = ( ( y_p - t.hosp)  / y_p).std() # relative error to the "true" curve

# Parameters for the base curve, the "true" values that we'll add error to. 
# Doesn't really matter what it is exactly, but should be similar to reality
curve_params = list(popt) + ([0]*(3-len(popt))) # Ensure there are always three components to the params

# Now we have the parameters to generate the "true" curve, and the std dev 
# to add errors to it. 

# curve_params, err_std



Now we can run build exponential curves with errors, fit them, and access the accuracy. Rather than analyze the errors in the fit parameters, we will predict the y value ( hosptlizations ) at day 60 and compare to the value for the "true" curve. 

In [9]:
# Shift from working with actual dates, to an undated series of values. 
start_day = (start_date - df.date.min()).days
end_day = (end_date - df.date.min()).days

cp = curve_params[:]
#cp[0] = 0

# Reset Initial curve fit parameters
func_exp.popt = curve_params[:2]
func_exp_c.popt = curve_params

def bootff(fit_func,start_day=start_day, end_day=end_day, err_std=err_std, curve_params=curve_params, iters=1000):

    # The two fit functions require different lengths of parameters
    ff_cp = curve_params[:len(fit_func.popt)]

    # Range of data that we'll fit to
    x_f = np.linspace(start_day, end_day) 
    y_f = fit_func(x_f,  *ff_cp)

    diff = None
    
    for i in range(iters):
        
        # Optimize zero error case
        if diff is not None and err_std == 0:
            yield diff
            continue
        
        # Add some noise
        y_noise = np.random.normal(0, err_std, size=len(y_f))
        y_e = y_f*(1+y_noise)

        popt, pcov = curve_fit(fit_func, x_f, y_e, p0=fit_func.popt, maxfev = 5000)

        # Take the difference between the predictions at day 60
        diff =  fit_func(60, *popt)
        yield diff

trials = pd.DataFrame({
    'exp2': list(bootff(func_exp, curve_params=cp)),
    'exp2_ze': list(bootff(func_exp, curve_params=cp, err_std=0)),
    'exp2_v': func_exp(60, *cp[:2]),
    'exp3': list(bootff(func_exp_c, curve_params=cp)),
    'exp3_ze': list(bootff(func_exp_c, curve_params=cp, err_std=0)),
    'exp3_v': func_exp_c(60, *cp),
})

trials['exp2_d'] = trials['exp2_v'] - trials['exp2']
trials['exp3_d'] = trials['exp3_v'] - trials['exp3']

tt =trials.describe()
tt

,exp2,exp2_ze,exp2_v,exp3,exp3_ze,exp3_v,exp2_d,exp3_d
count,1000.000000,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1000.000000,1.000000e+03
mean,20863.618459,1.713722e+04,1.713722e+04,3.740154e+05,1.711617e+04,1.711617e+04,-3726.401544,-3.568993e+05
std,12924.365541,3.639799e-12,3.639799e-12,2.080161e+06,3.639799e-12,3.639799e-12,12924.365541,2.080161e+06
min,3624.280148,1.713722e+04,1.713722e+04,2.615819e+02,1.711617e+04,1.711617e+04,-70223.712758,-3.359173e+07
25%,12118.628721,1.713722e+04,1.713722e+04,4.201817e+03,1.711617e+04,1.711617e+04,-8844.520363,-6.270487e+04
50%,17079.435937,1.713722e+04,1.713722e+04,1.886804e+04,1.711617e+04,1.711617e+04,57.780978,-1.751873e+03
75%,25981.737279,1.713722e+04,1.713722e+04,7.982104e+04,1.711617e+04,1.711617e+04,5018.588194,1.291435e+04
max,87360.929673,1.713722e+04,1.713722e+04,3.360885e+07,1.711617e+04,1.711617e+04,13512.936767,1.685459e+04


In [20]:
rows = [
    ['True Value', tt.loc['mean','exp2_v'], tt.loc['mean','exp3_v']],
    ['Prediction Mean', tt.loc['mean','exp2'], tt.loc['mean','exp3']],
    ['Prediction Std Dev', tt.loc['std','exp2'], tt.loc['std','exp3']],
    ['95 CI', tt.loc['std','exp2']*1.96, tt.loc['std','exp3']*1.96],
    
]

o = pd.DataFrame(rows, columns=['Metric','2 Parameter exp','3 parameter exp']).set_index('Metric')
o.astype(int)

,2 Parameter exp,3 parameter exp
Metric,,
True Value,17137,17116
Prediction Mean,20863,374015
Prediction Std Dev,12924,2080160
95 CI,25331,4077115
